In [1]:
# Import necessary packages
%matplotlib inline
import matplotlib.pyplot as plt #matplotlib
import numpy as np #numpy
import pandas as pd #pandas
import requests #requests
from bs4 import BeautifulSoup #BeautifulSoup
import datetime #datetime
import re #regularexpressions

In [2]:
# Read in the .csv file stored locally
nsidc_df = pd.read_csv('nsidc_df_rawscrape_2016-05-18_1123.csv')

In [3]:
nsidc_df.shape

(820, 22)

We were told by NSIDC that there were a few dataset_ids that corresponded to empty records. Below I will remove any rows for which all columns from 4 on are empty in the same row.

In [4]:
# remove rows that have no metadata from column 4 to the end of the columns
missing_metadata = nsidc_df.columns.values[4:20]
# show column names from 4:20
print missing_metadata
# Drop rows where all columns 4:20 contain NAs
nsidc_df.dropna(how='all', subset=missing_metadata, inplace=True)

['data_format_original' 'contributors_original' 'spatial_coverage_original'
 'spatial_resolution_original' 'temporal_coverage_original'
 'temporal_resolution_original' 'parameters_original' 'platforms_original'
 'sensors_original' 'version_original' 'doi_address' 'citation_date'
 'version_clean' 'location_original' 'keyword_original'
 'date_creation_original']


In [5]:
# reset the index after dropping rows, don't add another index column (drop=True)
nsidc_df = nsidc_df.reset_index(drop=True)

In [6]:
# Set new column width max
pd.set_option('max_colwidth',200)
# Check dataframe
nsidc_df.head(2)

,dataset_id,unique_users_ip,scrape_date,scrape_time,data_format_original,contributors_original,spatial_coverage_original,spatial_resolution_original,temporal_coverage_original,temporal_resolution_original,...,sensors_original,version_original,doi_address,citation_date,version_clean,location_original,keyword_original,date_creation_original,last_updated_original,title_original
0,g02186,24447,2016-05-18,10:45:22AM,\nPNG\nESRI Shapefile\nNetCDF\nMicrosoft Excel\nKeyhole Markup Language (.kml)\nASCII Text (.txt)\nGeoTIFF\n,"\n Florence Fetterer, Pablo Clemente-Colón, Matthew Savoie, Sean Helfrich","\nN: 90, S: 0, E: 180, W: -180\n\n",\n4 km x 4 km\n1 km x 1 km\n,\n1 October 2006\n (updated daily),\n 1 day,...,"\nAMSR-E, AMSU-A, AMSU-B, ASAR, AVHRR, GOES I-M IMAGER, MODIS, PALSAR, SAR, SEVIRI, SSM/I",\n V1,[[http://dx.doi.org/10.7265/N5GT5K3K]],[[2010]],1,Location: Arctic Ocean\nAtlantic Ocean\nNorth Atlantic Ocean\nBaltic Sea\nArctic Ocean\nBarents Sea\nBeaufort Sea\n,"[Arctic Sea Ice, ICE EDGE, Sea Ice, Sea Ice Conditions, Sea Ice Extent, Sea Ice Observations and Charts-project, Sea Ice PM Polar Stereo-project, Sea Ice Prediction Network-project, Sea Ice Visibl...",[[2010-12-14]],[[2016-02-09]],"Multisensor Analyzed Sea Ice Extent - Northern Hemisphere (MASIE-NH), Version 1"
1,g02135,22119,2016-05-18,10:45:22AM,\nPNG\nASCII Text (.txt)\nESRI Shapefile\n,"\n F. Fetterer, Kenneth Knowles, Walt Meier, Matthew Savoie","\nN: -39.23, S: -90, E: 180, W: -180\n\nN: 90, S: 30.98, E: 180, W: -180\n\n",\n25 km x 25 km\n,\n26 October 1978\n (updated daily),\n 1 day,...,"\nSMMR, SSM/I, SSMIS",\n V1,[[http://dx.doi.org/10.7265/N5QJ7F7W]],[[2002]],1,Location: Antarctica\nArctic\n,"[NOAA, Polar Stereo Sea Ice-project , Sea Ice, Sea Ice Anomalies, Sea Ice Conditions, Sea Ice Extent, Sea Ice PM Polar Stereo-project, Sea Ice PM Trends-project, Sea Ice Prediction Network-project...",[[2002-12-05]],[[2016-04-12]],"Sea Ice Index, Version 1"


In [7]:
for row in range(len(nsidc_df)):
    # Remove brackets from doi_address and citation_date and place results in new 'clean' column
    nsidc_df.loc[row, 'doi_address_clean'] = str(nsidc_df.loc[row, 'doi_address']).replace('[','').replace(']','')
    nsidc_df.loc[row, 'citation_date_clean'] = str(nsidc_df.loc[row, 'citation_date']).replace('[','').replace(']','')

In [8]:
# Set the data type for citation_date_clean as float for later data analysis
nsidc_df['citation_date_clean'] = nsidc_df['citation_date_clean'].astype('float')

In [9]:
# Check datatypes of all columns
nsidc_df.dtypes

dataset_id                       object
unique_users_ip                   int64
scrape_date                      object
scrape_time                      object
data_format_original             object
contributors_original            object
spatial_coverage_original        object
spatial_resolution_original      object
temporal_coverage_original       object
temporal_resolution_original     object
parameters_original              object
platforms_original               object
sensors_original                 object
version_original                 object
doi_address                      object
citation_date                    object
version_clean                   float64
location_original                object
keyword_original                 object
date_creation_original           object
last_updated_original            object
title_original                   object
doi_address_clean                object
citation_date_clean             float64
dtype: object

In [10]:
# Create a list containing the counts of data formats in each dataset
count_data_format = []

for element in nsidc_df['data_format_original']: 
    try:
        # Create string from everything in the element
        parts = " ".join(str(x) for x in nsidc_df['data_format_original'])
        # Split string into parts every time a \n is encountered
        parts = element.split('\n')
    except:
        pass
    # Append the count minus 2 (because the first and last element in the split are empty) 
    # to the list count_data_format.
    count_data_format.append((len(parts)-2))
# Add the count_data_format as a column to the two dataframes    
nsidc_df['count_data_format'] = count_data_format

In [11]:
nsidc_df.head(2)

,dataset_id,unique_users_ip,scrape_date,scrape_time,data_format_original,contributors_original,spatial_coverage_original,spatial_resolution_original,temporal_coverage_original,temporal_resolution_original,...,citation_date,version_clean,location_original,keyword_original,date_creation_original,last_updated_original,title_original,doi_address_clean,citation_date_clean,count_data_format
0,g02186,24447,2016-05-18,10:45:22AM,\nPNG\nESRI Shapefile\nNetCDF\nMicrosoft Excel\nKeyhole Markup Language (.kml)\nASCII Text (.txt)\nGeoTIFF\n,"\n Florence Fetterer, Pablo Clemente-Colón, Matthew Savoie, Sean Helfrich","\nN: 90, S: 0, E: 180, W: -180\n\n",\n4 km x 4 km\n1 km x 1 km\n,\n1 October 2006\n (updated daily),\n 1 day,...,[[2010]],1,Location: Arctic Ocean\nAtlantic Ocean\nNorth Atlantic Ocean\nBaltic Sea\nArctic Ocean\nBarents Sea\nBeaufort Sea\n,"[Arctic Sea Ice, ICE EDGE, Sea Ice, Sea Ice Conditions, Sea Ice Extent, Sea Ice Observations and Charts-project, Sea Ice PM Polar Stereo-project, Sea Ice Prediction Network-project, Sea Ice Visibl...",[[2010-12-14]],[[2016-02-09]],"Multisensor Analyzed Sea Ice Extent - Northern Hemisphere (MASIE-NH), Version 1",http://dx.doi.org/10.7265/N5GT5K3K,2010,7
1,g02135,22119,2016-05-18,10:45:22AM,\nPNG\nASCII Text (.txt)\nESRI Shapefile\n,"\n F. Fetterer, Kenneth Knowles, Walt Meier, Matthew Savoie","\nN: -39.23, S: -90, E: 180, W: -180\n\nN: 90, S: 30.98, E: 180, W: -180\n\n",\n25 km x 25 km\n,\n26 October 1978\n (updated daily),\n 1 day,...,[[2002]],1,Location: Antarctica\nArctic\n,"[NOAA, Polar Stereo Sea Ice-project , Sea Ice, Sea Ice Anomalies, Sea Ice Conditions, Sea Ice Extent, Sea Ice PM Polar Stereo-project, Sea Ice PM Trends-project, Sea Ice Prediction Network-project...",[[2002-12-05]],[[2016-04-12]],"Sea Ice Index, Version 1",http://dx.doi.org/10.7265/N5QJ7F7W,2002,3


In [12]:
# Create empty list
data_format_countvectorizer = []
# Loop through each element in data_format
for element in nsidc_df['data_format_original']: 
    try:
        # Create string from element
        parts = " ".join(str(x) for x in nsidc_df['data_format_original'])
        # Split string into parts at each \n
        parts = element.lower().split('\n')
        # Rejoin parts of element (without the \n)
        parts = " ".join(str(x) for x in parts)
        # Append all parts to list
        data_format_countvectorizer.append(parts)
    # If there was an error in the try code, append empty string and iterate
    except:
        data_format_countvectorizer.append('')
        pass
# Add a column to the countvectorizer dataframe with the contents of the data_format_countvectorizer list
nsidc_df['data_format_string'] = data_format_countvectorizer

In [13]:
for row in range(len(nsidc_df)):
    # Substitutions so that each data type is reduced to one word (in some cases using dashes to combine words)
    nsidc_df.loc[row, 'data_format_string'] = re.sub("ascii text \\(\\.txt\\)", "txt", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("keyhole markup language \\(\\.kml\\)", "kml", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("esri shapefile", "esri-shapefile", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("not specified", "not-specified", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("envi vector file \\(\\.evf\\)", "evf", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("generic mapping tools \\(\\.gmt\\)", "gmt", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("comma-separated values \\(\\.csv\\)", "csv", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("microsoft access", "microsoft-access", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("microsoft word", "microsoft-word", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("audio video interleave \\(\\.avi\\)", "avi", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("microsoft excel", "microsoft-excel", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("bitmap \\(\\.bmp\\)", "bmp", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("esri grid", "esri-grid", nsidc_df.loc[row, 'data_format_string'])
    nsidc_df.loc[row, 'data_format_string'] = re.sub("esri interchange", "esri-interchange", nsidc_df.loc[row, 'data_format_string'])

In [14]:
# Create binary variables for every unique data format
dummies_formats = nsidc_df['data_format_string'].str.get_dummies(sep=' ')

In [15]:
# Add dataset_id, unique_users_ip, scrape_date, and scrape_time to dummies_formats DB
dummies_formats['dataset_id'] = nsidc_df['dataset_id']
dummies_formats['unique_users_ip'] = nsidc_df['unique_users_ip']

# Change the order of the columns so that the four columns above appear first
cols = dummies_formats.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dummies_formats = dummies_formats[cols]

In [16]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save the platform binary features as csv
dummies_formats.to_csv('data_format_as_binaries_' +nowtime+ '.csv', index=False, encoding='utf-8')

In [17]:
# Clean contributors data - simplify group names
nsidc_df['contributors_clean'] = nsidc_df['contributors_original'].str.strip()
for row in range(len(nsidc_df)):
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("\\\n", "", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("\(comp\.\)\.", "", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("\(comps\.\)\.", "", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("National Snow and Ice Data Center", "NSIDC", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("National Snow and Ice Data Center\.", "NSIDC", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("National Snow and Ice Data Center", "NSIDC", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("NSIDC User Services", "NSIDC", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("National Ice Center\.", "NationalIceCenter", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("and National Snow and Ice Data Center", "NSIDC", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("Canadian Ice Service", "CanadianIceService", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("National Snow \& Ice Data Center", "NSIDC", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("and NSIDC", "NSIDC", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub(" Jr", "Jr", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub(" Sr", "Sr", nsidc_df.loc[row, 'contributors_clean'])
    nsidc_df.loc[row, 'contributors_clean'] = re.sub("III", "", nsidc_df.loc[row, 'contributors_clean'])

In [18]:
# More cleaning of contributors data
for row in range(len(nsidc_df)):
    nsidc_df.loc[row, 'contributors_clean'] = str(nsidc_df.loc[row, 'contributors_clean']).replace('"','')
    nsidc_df.loc[row, 'contributors_clean'] = str(nsidc_df.loc[row, 'contributors_clean']).replace(".",'')
    nsidc_df.loc[row, 'contributors_clean'] = str(nsidc_df.loc[row, 'contributors_clean']).replace("NSIDC ",'NSIDC')
    nsidc_df.loc[row, 'contributors_clean'] = str(nsidc_df.loc[row, 'contributors_clean']).replace(" and ",' ')
    nsidc_df.loc[row, 'contributors_clean'] = str(nsidc_df.loc[row, 'contributors_clean']).replace("O'Neill",'ONeill')

In [19]:
nsidc_df.tail(3)

,dataset_id,unique_users_ip,scrape_date,scrape_time,data_format_original,contributors_original,spatial_coverage_original,spatial_resolution_original,temporal_coverage_original,temporal_resolution_original,...,location_original,keyword_original,date_creation_original,last_updated_original,title_original,doi_address_clean,citation_date_clean,count_data_format,data_format_string,contributors_clean
794,nsidc-0654,1,2016-05-18,10:45:22AM,\nJPEG\nMicrosoft Excel\n,"\n NSIDC User Services, Eric Cravens","\nN: -79.467, S: -79.467, E: -112.085, W: -112.085\n\n",\n20 m\n,\n1 January 2008 to 30 August 2013\n,\n Not specified,...,Location: Antarctica\n,[AGDC-project],[[2016-03-07]],[[2016-03-07]],"WAIS Divide Ice Core Vertical Thin Section Low-resolution Digital Imagery, Version 1","http://dx.doi.org/10.7265/N5GM858X, http://dx.doi.org/10.3189/2014JoG14J100, http://dx/doi.org/10.3133/tm7D1",2016,2,jpeg microsoft-excel,"NSIDC, Eric Cravens"
795,nsidc-0655,1,2016-05-18,10:45:22AM,\nMicrosoft Excel\n,"\n Joan Fitzpatrick, Eric Cravens","\nN: -79.467, S: -79.467, E: -112.085, W: -112.085\n\n",\n20 m\n,\n1 January 2008 to 30 August 2013\n,\n Not specified,...,Location: Antarctica\n,[AGDC-project],[[2016-02-25]],[[2016-02-25]],"Grain Size Full Population Dataset from WDC06A Core, Version 1","http://dx.doi.org/10.7265/N5VX0DG0, http://dx.doi.org/10.3189/2014JoG14J100",2016,1,microsoft-excel,"Joan Fitzpatrick, Eric Cravens"
796,NMTHIR67-3H,1,2016-05-18,10:45:22AM,\nHDF\n,"\n David Gallaher, G. Garrett Campbell","\nN: 90, S: -90, E: 180, W: -180\n\n",\n10 km x 10 km\n20 km x 20 km\n,\n10 May 1970 to 25 March 1971\n,\n 1 day,...,Location: Global\n,[Nimbus-project],[[2015-11-24]],[[2015-11-24]],"Nimbus Temperature-Humidity Infrared Radiometer 6.7 µm Water Vapor Remapped Digital Data Daily, HDF5, Version 1",http://dx.doi.org/10.5067/NIMBUS/NmTHIR67-3H,2015,1,hdf,"David Gallaher, G Garrett Campbell"


In [20]:
# Create empty lists to append data
contributor_list = []
last_names = []

# Loop through cell in nsidc_df['contributors_clean']
for element in nsidc_df['contributors_clean']: 
    # Create empty lists to append data
    short_name = []
    lnames = []
    try:
        # Create string from everything in the element
        parts = " ".join(str(x) for x in nsidc_df['contributors_clean'])
        # Split string into parts every time a , is encountered
        parts = element.split(', ')
        # Loop through each name in parts
        for name in parts:
            fullname = name.split(' ')
            last_name = fullname[-1]
            #print last_name
            if len(fullname) > 1:
                first_initial = name[0][:1]
                short_name.append(first_initial +"-"+ last_name)
            else:
                short_name.append(last_name)
            
            lnames.append(last_name)
            
        # append the first item in parts to contributor_1
        #print short_name
        contributor_list.append(short_name)
        last_names.append(lnames)
    except:
        pass
nsidc_df['contributor_list'] = contributor_list
nsidc_df['contributor_last_names'] = last_names

In [21]:
# Fix the fact that the data within contributor_list and contibutor_last_names are currently lists
for row in range(len(nsidc_df['contributor_list'])):
    unlist1 = ", ".join(str(x) for x in nsidc_df['contributor_list'][row])
    nsidc_df.loc[row, 'contributor_list'] = unlist1
    unlist2 = ", ".join(str(x) for x in nsidc_df['contributor_last_names'][row])
    nsidc_df.loc[row, 'contributor_last_names'] = unlist2

nsidc_df[11:15]

,dataset_id,unique_users_ip,scrape_date,scrape_time,data_format_original,contributors_original,spatial_coverage_original,spatial_resolution_original,temporal_coverage_original,temporal_resolution_original,...,date_creation_original,last_updated_original,title_original,doi_address_clean,citation_date_clean,count_data_format,data_format_string,contributors_clean,contributor_list,contributor_last_names
11,g01130,876,2016-05-18,10:45:22AM,\nASCII Text (.txt)\n,"\n WGMS, and National Snow and Ice Data Center (comps.).","\nN: 89.3, S: -71.66, E: 180, W: -180\n\n",\n Not Specified,\n1 January 1900 to 31 December 2003\n (updated 2012),\n Not specified,...,[[1999-01-01]],[[2015-09-09]],"World Glacier Inventory, Version 1",http://dx.doi.org/10.7265/N5/NSIDC-WGI-2012-02,1999,1,txt,"WGMS, NSIDC","WGMS, -NSIDC","WGMS, NSIDC"
12,MYD10A1,675,2016-05-18,10:45:22AM,\nHDF-EOS\n,"\n Miguel Román, Dorothy Hall, George Riggs","\nN: 90, S: -90, E: 180, W: -180\n\n",\n500 m x 500 m\n,\n4 July 2002\n,\n 1 day,...,[[2016-04-25]],[[2016-04-25]],"MODIS/Aqua Snow Cover Daily L3 Global 500m Grid, Version 6","http://dx.doi.org/10.5067/MODIS/MYD10A1.006, http://dx.doi.org/10.1007/978-90-481-2642-2_376, http://dx.doi.org/10.1029/2012GL053387, http://dx.doi.org/10.1109/LGRS.2011.2180505., http://dx.doi.or...",2016,1,hdf-eos,"Miguel Román, Dorothy Hall, George Riggs","M-Román, D-Hall, G-Riggs","Román, Hall, Riggs"
13,nsidc-0102,649,2016-05-18,10:45:22AM,\n Not specified,"\n Ted Scambos, Bruce Raup, Jennifer Bohlander",\n Not specified,\n Not Specified,\n Not specified,\n Not specified,...,[[2001-01-01]],[[2014-03-14]],"Images of Antarctic Ice Shelves, Version 1",http://dx.doi.org/10.7265/N5NC5Z4N,2001,0,not-specified,"Ted Scambos, Bruce Raup, Jennifer Bohlander","T-Scambos, B-Raup, J-Bohlander","Scambos, Raup, Bohlander"
14,NISE,635,2016-05-18,10:45:22AM,\nHDF-EOS\n,"\n Anne Nolin, Richard Armstrong, Jim Maslanik","\nN: 90, S: -90, E: 180, W: -180\n\n",\n Not Specified,\n4 May 1995\n,\n 1 day,...,[[1998-01-01]],[[2016-04-21]],"Near-Real-Time SSM/I-SSMIS EASE-Grid Daily Global Ice Concentration and Snow Extent, Version 4",http://dx.doi.org/10.5067/VF7QO90IHZ99,1998,1,hdf-eos,"Anne Nolin, Richard Armstrong, Jim Maslanik","A-Nolin, R-Armstrong, J-Maslanik","Nolin, Armstrong, Maslanik"


In [22]:
# Create binary variables for every unique last name
dummies_last_names = nsidc_df['contributor_last_names'].str.get_dummies(sep=', ')
# Create binary variables for every unique first inital last name
dummies_contributor_list = nsidc_df['contributor_list'].str.get_dummies(sep=', ')

In [23]:
# Find out how many items are in contributor list and how many items are unique
thelist=[]
for element in nsidc_df['contributor_list']: 
    # Create string from everything in the element
    parts = " ".join(str(x) for x in nsidc_df['contributor_list'])
    # Split string into parts every time a , is encountered
    parts = element.split(', ')
    for name in parts:
        thelist.append(name)
print len(thelist)
print len(set(thelist))

2057
772


In [24]:
# Check the shape
dummies_last_names.shape

(797, 712)

In [25]:
# Check the shape
dummies_contributor_list.shape

(797, 771)

It is reasonable that the shape of dummies_last_names and dummies_contributor_list don't match. There may be contributors who share the same last name but not the first initial.

In [26]:
# Add dataset_id, unique_users_ip, scrape_date, and scrape_time to dummies_last_names DB
dummies_last_names['dataset_id'] = nsidc_df['dataset_id']
dummies_last_names['unique_users_ip'] = nsidc_df['unique_users_ip']

# Rearrange order of columns
cols = dummies_last_names.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dummies_last_names = dummies_last_names[cols]

In [27]:
# Add dataset_id, unique_users_ip, scrape_date, and scrape_time to dummies_last_names DB
dummies_contributor_list['dataset_id'] = nsidc_df['dataset_id']
dummies_contributor_list['unique_users_ip'] = nsidc_df['unique_users_ip']

# Rearrange order of columns
cols = dummies_contributor_list.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dummies_contributor_list = dummies_contributor_list[cols]

In [28]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save the platform binary features as csv
dummies_last_names.to_csv('last_names_as_binaries_' +nowtime+ '.csv', index=False, encoding='utf-8')
dummies_contributor_list.to_csv('contributor_list_as_binaries_' +nowtime+ '.csv', index=False, encoding='utf-8')

In [29]:
# Create empty lists to fill with count of contributors per dataset
count_contributors = []

for element in nsidc_df['contributors_original']: 
    try:
        # Create string from everything in the element
        parts = " ".join(str(x) for x in nsidc_df['contributors_original'])
        # Split string into parts every time a , is encountered
        parts = element.split(',')
    except:
        pass
    count_contributors.append(len(parts))
nsidc_df['count_contributors'] = count_contributors

In [30]:
# Find out which contirbutor appears most often
result = dummies_contributor_list.apply(pd.value_counts).fillna(0)
result.ix[1].idxmax()

'T-Jackson'

In [31]:
# Create empty lists to fill with count of spatial coverage units per dataset
count_spatial_coverage = []
spatial_coverage_list = []

for element in nsidc_df['spatial_coverage_original']: 
    try:
        # Create string from everything in the element
        #parts = " ".join(str(x) for x in nsidc_df['spatial_coverage_clean'])
        # Split string into parts every time a , is encountered
        parts = element.split('\n\n')
        cnt = len(parts)
        # Rejoin parts of element (without the \n)
        parts = ";".join(str(x) for x in parts)
        # Append all parts to list
        spatial_coverage_list.append(parts)
    except:
        spatial_coverage_list.append('')
        pass
    count_spatial_coverage.append(cnt-1)
nsidc_df['spatial_coverage_clean'] = spatial_coverage_list
nsidc_df['count_spatial_coverage'] = count_spatial_coverage

In [32]:
nsidc_df['spatial_coverage_clean'] = nsidc_df['spatial_coverage_clean'].str.strip('\n')

In [33]:
# Create empty lists to fill with spatial coverage
spatial_coverage_1 = []
spatial_coverage_2 = []
spatial_coverage_3 = []
spatial_coverage_4 = []
spatial_coverage_5 = []
spatial_coverage_6 = []
spatial_coverage_7 = []
spatial_coverage_8 = []
spatial_coverage_9 = []

# Set maxel = 0, this will update in the loop until it reaches the max length of "parts"
for element in nsidc_df['spatial_coverage_clean']: 
    try:
        #  Create string from everything in the element
        parts = " ".join(str(x) for x in nsidc_df['spatial_coverage_clean'])
        parts = element.split(';')
        if maxel < len(parts):
            maxel = len(parts)
        spatial_coverage_1.append(parts[1])
        # If there was an error in the try code, append the string 'NULL' to the array prereq, update i, iterate
    except:
        spatial_coverage_1.append('')
        pass
    try:
        spatial_coverage_2.append(parts[2])
    except:
        spatial_coverage_2.append('')
        pass
    try:
        spatial_coverage_3.append(parts[3])
    except:
        spatial_coverage_3.append('')
        pass
    try:
        spatial_coverage_4.append(parts[4])
    except:
        spatial_coverage_4.append('')
        pass
    try:
        spatial_coverage_5.append(parts[5])
    except:
        spatial_coverage_5.append('')
        pass
    try:
        spatial_coverage_6.append(parts[6])
    except:
        spatial_coverage_6.append('')
        pass
    try:
        spatial_coverage_7.append(parts[7])
    except:
        spatial_coverage_7.append('')
        pass
    try:
        spatial_coverage_8.append(parts[8])
    except:
        spatial_coverage_8.append('')
        pass
    try:
        spatial_coverage_9.append(parts[9])
    except:
        spatial_coverage_9.append('')
        pass

In [34]:
nsidc_df_spatialcov = pd.DataFrame(index=nsidc_df.index)

# Assign parts of spatial coverage to individual columns
nsidc_df_spatialcov['dataset_id'] = nsidc_df['dataset_id']
nsidc_df_spatialcov['unique_users_ip'] = nsidc_df['unique_users_ip']
nsidc_df_spatialcov['spatial_coverage_1'] = spatial_coverage_1
#nsidc_df_spatialcov['spatial_coverage_2'] = spatial_coverage_2
nsidc_df_spatialcov['spatial_coverage_2'] = spatial_coverage_3
#nsidc_df_spatialcov['spatial_coverage_4'] = spatial_coverage_4
nsidc_df_spatialcov['spatial_coverage_3'] = spatial_coverage_5
#nsidc_df_spatialcov['spatial_coverage_6'] = spatial_coverage_6
nsidc_df_spatialcov['spatial_coverage_4'] = spatial_coverage_7
#nsidc_df_spatialcov['spatial_coverage_8'] = spatial_coverage_8
nsidc_df_spatialcov['spatial_coverage_5'] = spatial_coverage_9

In [35]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save the platform binary features as csv
nsidc_df_spatialcov.to_csv('spatial_coverage_' +nowtime+ '.csv', index=False, encoding='utf-8')

In [36]:

# Create empty list
spatial_resolution_list = []
count_spatial_resolution = []
# Loop through each element in data_format
for element in nsidc_df['spatial_resolution_original']: 
    try:
        # Create string from element
        #parts = " ".join(str(x) for x in nsidc_df['spatial_resolution_orginal'])
        # Split string into parts at each \n
        parts = element.split('\n')
        
        cnt = len(parts)
        # Rejoin parts of element (without the \n)
        parts = ",".join(str(x) for x in parts)
        # Append all parts to list
        spatial_resolution_list.append(parts)
    # If there was an error in the try code, append empty string and iterate
    except:
        spatial_resolution_list.append('')
        pass
    count_spatial_resolution.append(cnt-2)
# Add a column to the countvectorizer dataframe with the contents of the data_format_countvectorizer list
nsidc_df['spatial_resolution_clean'] = spatial_resolution_list
nsidc_df['count_spatial_resolution'] = count_spatial_resolution

In [37]:
nsidc_df['spatial_resolution_clean'] = nsidc_df['spatial_resolution_clean'].str.strip(',')

In [38]:
nsidc_df[['spatial_coverage_original', 'count_spatial_coverage', 'spatial_coverage_clean', 'spatial_resolution_original', 'spatial_resolution_clean', 'count_spatial_resolution']].head(3)

,spatial_coverage_original,count_spatial_coverage,spatial_coverage_clean,spatial_resolution_original,spatial_resolution_clean,count_spatial_resolution
0,"\nN: 90, S: 0, E: 180, W: -180\n\n",1,"N: 90, S: 0, E: 180, W: -180;",\n4 km x 4 km\n1 km x 1 km\n,"4 km x 4 km,1 km x 1 km,",2
1,"\nN: -39.23, S: -90, E: 180, W: -180\n\nN: 90, S: 30.98, E: 180, W: -180\n\n",2,"N: -39.23, S: -90, E: 180, W: -180;N: 90, S: 30.98, E: 180, W: -180;",\n25 km x 25 km\n,"25 km x 25 km,",1
2,"\nN: 81, S: -43, E: 180, W: -180\n\n",1,"N: 81, S: -43, E: 180, W: -180;",\n Not Specified,Not Specified,0


In [39]:
nsidc_df.iloc[0:10,30:]

,spatial_coverage_clean,count_spatial_coverage,spatial_resolution_clean,count_spatial_resolution
0,"N: 90, S: 0, E: 180, W: -180;",1,"4 km x 4 km,1 km x 1 km,",2
1,"N: -39.23, S: -90, E: 180, W: -180;N: 90, S: 30.98, E: 180, W: -180;",2,"25 km x 25 km,",1
2,"N: 81, S: -43, E: 180, W: -180;",1,Not Specified,0
3,"N: 90, S: 30.98, E: 180, W: -180;N: -39.23, S: -90, E: 180, W: -180;",2,"25 km x 25 km,",1
4,"N: -39.23, S: -90, E: 180, W: -180;N: 90, S: 30.98, E: 180, W: -180;",2,"25 km x 25 km,",1
5,"N: 90, S: -90, E: 180, W: -180;",1,"500 m x 500 m,",1
6,"N: 90, S: 0, E: 180, W: -180;",1,"1 km x 1 km,4 km x 4 km,24 km x 24 km,",3
7,"N: 58.2329, S: 24.0996, E: -62.2504, W: -130.5171;N: 52.8754, S: 24.9504, E: -66.9421, W: -124.7337;",2,"1 km x 1 km,",1
8,"N: -39.23, S: -90, E: 180, W: -180;N: 90, S: 30.98, E: 180, W: -180;",2,"25 km x 25 km,",1
9,"N: 89.84, S: 31.1, E: 180, W: -180;N: -39.36, S: -89.84, E: 180, W: -180;",2,"25 km x 25 km,",1


In [40]:
for row in range(len(nsidc_df)):
    # Get rid of \n
    nsidc_df.loc[row, 'platforms_clean'] = re.sub("\\\n", "", nsidc_df.loc[row, 'platforms_original'])

# strip white spaces to the left and right of strings
nsidc_df['platforms_clean'] = nsidc_df['platforms_clean'].str.strip()

In [41]:
# Create empty list
platforms_list = []
count_platforms = []
# Loop through each element in data_format
for element in nsidc_df['platforms_clean']: 
    try:
        # Create string from element
        parts = " ".join(str(x) for x in nsidc_df['platforms_clean'])
        # Split string into parts at each ,
        parts = element.lower().split(', ')
        #parts = parts.str.strip()
        cnt = len(parts)
        # Rejoin parts of element (without the \n)
        parts = ",".join(str(x) for x in parts)
        # Append all parts to list
        platforms_list.append(parts)
    # If there was an error in the try code, append empty string and iterate
    except:
        platforms_list.append('')
        pass
    count_platforms.append(cnt)
# Add a column to the countvectorizer dataframe with the contents of the data_format_countvectorizer list
nsidc_df['platforms_clean'] = platforms_list
nsidc_df['count_platforms'] = count_platforms

In [42]:
for i in range(len(nsidc_df)): 
    if nsidc_df.loc[i, 'platforms_clean'] == 'not specified':
        nsidc_df.loc[i, 'count_platforms'] = 0

In [43]:
# Create binary variables for every unique platform
dummies_platform = nsidc_df['platforms_clean'].str.get_dummies(sep=',')

In [44]:
# Add columns to dummies_platform dataframe
dummies_platform['dataset_id'] = nsidc_df['dataset_id']
dummies_platform['unique_users_ip'] = nsidc_df['unique_users_ip']

# Rearrange order of columns
cols = dummies_platform.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dummies_platform = dummies_platform[cols]

In [45]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save the platform binary features as csv
dummies_platform.to_csv('platforms_as_binaries_' +nowtime+ '.csv', index=False, encoding='utf-8')

In [46]:
for row in range(len(nsidc_df)):
    # Get rid of \n
    nsidc_df.loc[row, 'sensors_clean'] = re.sub("\\\n", "", nsidc_df.loc[row, 'sensors_original'])

# strip white spaces to the left and right of strings
nsidc_df['sensors_clean'] = nsidc_df['sensors_clean'].str.strip()

In [47]:
# Create empty list
sensors_list = []
count_sensors = []
# Loop through each element in data_format
for element in nsidc_df['sensors_clean']: 
    try:
        # Create string from element
        parts = " ".join(str(x) for x in nsidc_df['sensors_clean'])
        # Split string into parts at each ,
        parts = element.lower().split(', ')
        #parts = parts.str.strip()
        cnt = len(parts)
        # Rejoin parts of element (without the \n)
        parts = ",".join(str(x) for x in parts)
        # Append all parts to list
        sensors_list.append(parts)
    # If there was an error in the try code, append empty string and iterate
    except:
        sensors_list.append('')
        pass
    count_sensors.append(cnt)
# Add a column to the countvectorizer dataframe with the contents of the data_format_countvectorizer list
nsidc_df['sensors_clean'] = sensors_list
nsidc_df['count_sensors'] = count_sensors

In [48]:
for i in range(len(nsidc_df)): 
    if nsidc_df.loc[i, 'sensors_clean'] == 'not specified':
        nsidc_df.loc[i, 'count_sensors'] = 0

In [49]:
# Create binary variables for every unique sensor
dummies_sensor = nsidc_df['sensors_clean'].str.get_dummies(sep=',')

In [50]:
# Add dataset_id, unique_users_ip, scrape_date, and scrape_time to dummies_sensor DB
dummies_sensor['dataset_id'] = nsidc_df['dataset_id']
dummies_sensor['unique_users_ip'] = nsidc_df['unique_users_ip']

# Rearrange order of columns
cols = dummies_sensor.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dummies_sensor = dummies_sensor[cols]

In [51]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save the sensor binary features as csv
dummies_sensor.to_csv('sensors_as_binaries_' +nowtime+ '.csv', index=False, encoding='utf-8')

In [52]:
# Create an empty list
update = []
# Set i = 0 for while loop
i=0
# Loop through rows in nsidc_df
while i < len(nsidc_df):
    try:
        # Remove \n from items in temporal_coverage_original
        thing = re.sub("\\\n", "", nsidc_df.loc[i, 'temporal_coverage_original'])
        # create a regex object to compile everything that follows the word updated
        regexp = re.compile("updated(.*)$", re.I)
        str1 = regexp.search(thing).group(1)
        if str1 == "None":
            update.append("")
        else:
            update.append(str1.split(")")[0])
        i+=1
    # If there was an error in the try code, append and empty string, update i, iterate
    except:
        update.append("")
        i+=1
        pass
    
# Add column to nsidc_df with new update data
nsidc_df['update_frequency'] = update

In [53]:
for row in range(len(nsidc_df)):
    # Get rid of .
    nsidc_df.loc[row, 'update_frequency'] = re.sub("\\.", "", nsidc_df.loc[row, 'update_frequency'])
    # annually and yearly have the same meaning - replace all annually occurrences with yearly
    nsidc_df.loc[row, 'update_frequency'] = re.sub("annually", "yearly", nsidc_df.loc[row, 'update_frequency'])
nsidc_df['update_frequency'] = nsidc_df['update_frequency'].str.strip()

In [54]:
# Create binary variables for every unique sensor
dummies_update_frequency = nsidc_df['update_frequency'].str.get_dummies()

In [55]:
dummies_update_frequency.head(5)

,1990,1996,2004,2005,2006,2008,2009,2012,2013,2014,2015,2016,January 2009,daily,yearly
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [56]:
# Add dataset_id, unique_users_ip, scrape_date, and scrape_time 
dummies_update_frequency['dataset_id'] = nsidc_df['dataset_id']
dummies_update_frequency['unique_users_ip'] = nsidc_df['unique_users_ip']

# Rearrange order of columns
cols = dummies_update_frequency.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dummies_update_frequency = dummies_update_frequency[cols]

In [57]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save the update frequency binary features as csv
dummies_update_frequency.to_csv('update_frequency_as_binaries_' +nowtime+ '.csv', index=False, encoding='utf-8')

In [58]:
nsidc_df['location_clean'] = nsidc_df['location_original'].astype('str')
for row in range(len(nsidc_df)):
    nsidc_df.loc[row, 'location_clean'] = str(nsidc_df.loc[row, 'location_clean']).replace('\xa0','')
    nsidc_df.loc[row, 'location_clean'] = str(nsidc_df.loc[row, 'location_original']).replace(':','')
    nsidc_df.loc[row, 'location_clean'] = str(nsidc_df.loc[row, 'location_clean']).replace('Location','')
    nsidc_df.loc[row, 'location_clean'] = nsidc_df.loc[row, 'location_clean'].decode('utf-8', errors='ignore')
nsidc_df['location_clean'] = nsidc_df['location_clean'].str.strip()

In [59]:
# Create empty list
location_list = []
count_locations = []
# Loop through each element in data_format
for element in nsidc_df['location_clean']: 
    try:
        # Create string from element
        parts = " ".join(str(x) for x in nsidc_df['location_clean'])
        # Split string into parts at each \n
        parts = element.lower().split('\n')
        cnt = len(parts)
        # Rejoin parts of element (without the \n)
        parts = ",".join(str(x) for x in parts)
        # Append all parts to list
        location_list.append(parts)
    # If there was an error in the try code, append empty string and iterate
    except:
        location_list.append('')
        pass
    count_locations.append(cnt)
# Add a column to the countvectorizer dataframe with the contents of the data_format_countvectorizer list
nsidc_df['location_clean'] = location_list
nsidc_df['count_locations'] = count_locations

In [60]:
for i in range(len(nsidc_df)): 
    leng = len(nsidc_df.loc[i, 'location_clean'])
    if leng == 0:
        nsidc_df.loc[i, 'count_locations'] = leng

In [61]:
dummies_location = nsidc_df['location_clean'].str.get_dummies(sep=',')

In [62]:
# Add dataset_id, unique_users_ip, scrape_date, and scrape_time to dummies_last_names DB
dummies_location['dataset_id'] = nsidc_df['dataset_id']
dummies_location['unique_users_ip'] = nsidc_df['unique_users_ip']

# Rearrange order of columns
cols = dummies_location.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dummies_location = dummies_location[cols]

In [63]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save the update frequency binary features as csv
dummies_location.to_csv('location_as_binaries_' +nowtime+ '.csv', index=False, encoding='utf-8')

In [64]:
# Create empty list
keyword_list = []
count_keyword = []
# Loop through each element in data_format
for element in nsidc_df['keyword_original']: 
    try:
        # Create string from element
        parts = " ".join(str(x) for x in nsidc_df['keyword_original'])
        # Split string into parts at each ,
        parts = element.lower().split(', ')
        cnt = len(parts)
        # Rejoin parts of element (without the \n)
        parts = ",".join(str(x) for x in parts)
        # Append all parts to list
        keyword_list.append(parts)
    # If there was an error in the try code, append empty string and iterate
    except:
        keyword_list.append('')
        pass
    count_keyword.append(cnt)
# Add a column to the countvectorizer dataframe with the contents of the data_format_countvectorizer list
nsidc_df['keyword_clean'] = keyword_list
nsidc_df['count_keyword'] = count_keyword

In [65]:
for i in range(len(nsidc_df)): 
    leng = len(nsidc_df.loc[i, 'keyword_clean'])
    if leng == 0:
        nsidc_df.loc[i, 'count_keyword'] = leng

In [66]:
nsidc_df.head(2)

,dataset_id,unique_users_ip,scrape_date,scrape_time,data_format_original,contributors_original,spatial_coverage_original,spatial_resolution_original,temporal_coverage_original,temporal_resolution_original,...,count_spatial_resolution,platforms_clean,count_platforms,sensors_clean,count_sensors,update_frequency,location_clean,count_locations,keyword_clean,count_keyword
0,g02186,24447,2016-05-18,10:45:22AM,\nPNG\nESRI Shapefile\nNetCDF\nMicrosoft Excel\nKeyhole Markup Language (.kml)\nASCII Text (.txt)\nGeoTIFF\n,"\n Florence Fetterer, Pablo Clemente-Colón, Matthew Savoie, Sean Helfrich","\nN: 90, S: 0, E: 180, W: -180\n\n",\n4 km x 4 km\n1 km x 1 km\n,\n1 October 2006\n (updated daily),\n 1 day,...,2,"alos,aqua,dmsp,envisat,ers-2,goes,msg,noaa poes,radarsat-2,satellites",10,"amsr-e,amsu-a,amsu-b,asar,avhrr,goes i-m imager,modis,palsar,sar,seviri,ssm/i",11,daily,"arctic ocean,atlantic ocean,north atlantic ocean,baltic sea,arctic ocean,barents sea,beaufort sea",7,"[arctic sea ice,ice edge,sea ice,sea ice conditions,sea ice extent,sea ice observations and charts-project,sea ice pm polar stereo-project,sea ice prediction network-project,sea ice visible/infrar...",9
1,g02135,22119,2016-05-18,10:45:22AM,\nPNG\nASCII Text (.txt)\nESRI Shapefile\n,"\n F. Fetterer, Kenneth Knowles, Walt Meier, Matthew Savoie","\nN: -39.23, S: -90, E: 180, W: -180\n\nN: 90, S: 30.98, E: 180, W: -180\n\n",\n25 km x 25 km\n,\n26 October 1978\n (updated daily),\n 1 day,...,1,"dmsp,dmsp 5d-3/f17,nimbus-7,satellites",4,"smmr,ssm/i,ssmis",3,daily,"antarctica,arctic",2,"[noaa,polar stereo sea ice-project ,sea ice,sea ice anomalies,sea ice conditions,sea ice extent,sea ice pm polar stereo-project,sea ice pm trends-project,sea ice prediction network-project,sea ice...",10


In [67]:
for row in range(len(nsidc_df)):
    nsidc_df.loc[row, 'keyword_clean'] = str(nsidc_df.loc[row, 'keyword_clean']).replace(']','')
    nsidc_df.loc[row, 'keyword_clean'] = str(nsidc_df.loc[row, 'keyword_clean']).replace('[','')

In [68]:
nsidc_df['keyword_clean'] = nsidc_df['keyword_clean'].str.strip()
dummies_keyword = nsidc_df['keyword_clean'].str.get_dummies(sep=',')

In [69]:
# Add dataset_id, unique_users_ip, scrape_date, and scrape_time to dummies_last_names DB
dummies_keyword['dataset_id'] = nsidc_df['dataset_id']
dummies_keyword['unique_users_ip'] = nsidc_df['unique_users_ip']

# Rearrange order of columns
cols = dummies_keyword.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dummies_keyword = dummies_keyword[cols]

In [70]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save the update frequency binary features as csv
dummies_keyword.to_csv('keyword_as_binaries_' +nowtime+ '.csv', index=False, encoding='utf-8')

In [71]:
nsidc_df.head(2)

,dataset_id,unique_users_ip,scrape_date,scrape_time,data_format_original,contributors_original,spatial_coverage_original,spatial_resolution_original,temporal_coverage_original,temporal_resolution_original,...,count_spatial_resolution,platforms_clean,count_platforms,sensors_clean,count_sensors,update_frequency,location_clean,count_locations,keyword_clean,count_keyword
0,g02186,24447,2016-05-18,10:45:22AM,\nPNG\nESRI Shapefile\nNetCDF\nMicrosoft Excel\nKeyhole Markup Language (.kml)\nASCII Text (.txt)\nGeoTIFF\n,"\n Florence Fetterer, Pablo Clemente-Colón, Matthew Savoie, Sean Helfrich","\nN: 90, S: 0, E: 180, W: -180\n\n",\n4 km x 4 km\n1 km x 1 km\n,\n1 October 2006\n (updated daily),\n 1 day,...,2,"alos,aqua,dmsp,envisat,ers-2,goes,msg,noaa poes,radarsat-2,satellites",10,"amsr-e,amsu-a,amsu-b,asar,avhrr,goes i-m imager,modis,palsar,sar,seviri,ssm/i",11,daily,"arctic ocean,atlantic ocean,north atlantic ocean,baltic sea,arctic ocean,barents sea,beaufort sea",7,"arctic sea ice,ice edge,sea ice,sea ice conditions,sea ice extent,sea ice observations and charts-project,sea ice pm polar stereo-project,sea ice prediction network-project,sea ice visible/infrare...",9
1,g02135,22119,2016-05-18,10:45:22AM,\nPNG\nASCII Text (.txt)\nESRI Shapefile\n,"\n F. Fetterer, Kenneth Knowles, Walt Meier, Matthew Savoie","\nN: -39.23, S: -90, E: 180, W: -180\n\nN: 90, S: 30.98, E: 180, W: -180\n\n",\n25 km x 25 km\n,\n26 October 1978\n (updated daily),\n 1 day,...,1,"dmsp,dmsp 5d-3/f17,nimbus-7,satellites",4,"smmr,ssm/i,ssmis",3,daily,"antarctica,arctic",2,"noaa,polar stereo sea ice-project ,sea ice,sea ice anomalies,sea ice conditions,sea ice extent,sea ice pm polar stereo-project,sea ice pm trends-project,sea ice prediction network-project,sea ice ...",10


In [72]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save dataframes to csv
nsidc_df.to_csv('nsidc_df_' + nowtime + '.csv', index=False, encoding='utf-8')

In [73]:
dropcolumns = nsidc_df.columns[4:16]
nsidc_clean_df = nsidc_df.drop(dropcolumns, axis=1)
nsidc_clean_df.columns

Index([u'dataset_id', u'unique_users_ip', u'scrape_date', u'scrape_time',
       u'version_clean', u'location_original', u'keyword_original',
       u'date_creation_original', u'last_updated_original', u'title_original',
       u'doi_address_clean', u'citation_date_clean', u'count_data_format',
       u'data_format_string', u'contributors_clean', u'contributor_list',
       u'contributor_last_names', u'count_contributors',
       u'spatial_coverage_clean', u'count_spatial_coverage',
       u'spatial_resolution_clean', u'count_spatial_resolution',
       u'platforms_clean', u'count_platforms', u'sensors_clean',
       u'count_sensors', u'update_frequency', u'location_clean',
       u'count_locations', u'keyword_clean', u'count_keyword'],
      dtype='object')

In [74]:
dropcolumns = nsidc_clean_df.columns[5:9]
nsidc_clean_df = nsidc_clean_df.drop(dropcolumns, axis=1)
nsidc_clean_df.columns

Index([u'dataset_id', u'unique_users_ip', u'scrape_date', u'scrape_time',
       u'version_clean', u'title_original', u'doi_address_clean',
       u'citation_date_clean', u'count_data_format', u'data_format_string',
       u'contributors_clean', u'contributor_list', u'contributor_last_names',
       u'count_contributors', u'spatial_coverage_clean',
       u'count_spatial_coverage', u'spatial_resolution_clean',
       u'count_spatial_resolution', u'platforms_clean', u'count_platforms',
       u'sensors_clean', u'count_sensors', u'update_frequency',
       u'location_clean', u'count_locations', u'keyword_clean',
       u'count_keyword'],
      dtype='object')

In [75]:
# Capture the current time to add to the csv file name
nowtime = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
# Save dataframes to csv
nsidc_clean_df.to_csv('nsidc_clean_df_' + nowtime + '.csv', index=False, encoding='utf-8')

In [84]:
nsidc_df[['dataset_id', 'unique_users_ip', 'data_format_original', 'data_format_string', 'count_data_format']]

,dataset_id,unique_users_ip,data_format_original,data_format_string,count_data_format
0,g02186,24447,\nPNG\nESRI Shapefile\nNetCDF\nMicrosoft Excel\nKeyhole Markup Language (.kml)\nASCII Text (.txt)\nGeoTIFF\n,png esri-shapefile netcdf microsoft-excel kml txt geotiff,7
1,g02135,22119,\nPNG\nASCII Text (.txt)\nESRI Shapefile\n,png txt esri-shapefile,3
2,g00472,5862,\nTIFF\nJPEG\n,tiff jpeg,2
3,nsidc-0081,5629,\nPNG\nBinary\n,png binary,2
4,nsidc-0051,4526,\nBinary\nPNG\n,binary png,2
5,MOD10A1,1461,\nHDF-EOS\n,hdf-eos,1
6,g02156,1425,\nGIF\nASCII Text (.txt)\nGeoTIFF\n,gif txt geotiff,3
7,g02158,1389,\nBinary\n,binary,1
8,nsidc-0192,1235,\nBinary\nPNG\nASCII Text (.txt)\nJPEG\n,binary png txt jpeg,4
9,g02202,992,\nNetCDF\n,netcdf,1


In [83]:
dummies_formats

,dataset_id,unique_users_ip,arcgis,avi,binary,bmp,csv,dxf,esri-grid,esri-interchange,...,photoshop,png,segy,sir,tiff,twf,txt,wfs,wms,xml
0,g02186,24447,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,g02135,22119,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,g00472,5862,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,nsidc-0081,5629,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,nsidc-0051,4526,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,MOD10A1,1461,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,g02156,1425,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,g02158,1389,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,nsidc-0192,1235,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
9,g02202,992,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
